In [1]:
import pandas as pd
import numpy as np




In [2]:
print("Reading the filtered dataset...")
df = pd.read_csv('./borg_traces_data.csv')


Reading the filtered dataset...


C:\Users\nithi\AppData\Local\Temp\ipykernel_11748\155152889.py:2: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./borg_traces_data.csv')


In [3]:
# Basic Information
print("\n=== Basic Dataset Information ===")
print("Dataset Shape:", df.shape)
print("\nNumber of columns:", len(df.columns))
print("\nColumn names:")
for col in df.columns:
    print(f"- {col}, {df[col].dtype}")





=== Basic Dataset Information ===
Dataset Shape: (405894, 34)

Number of columns: 34

Column names:
- Unnamed: 0, int64
- time, int64
- instance_events_type, int64
- collection_id, int64
- scheduling_class, int64
- collection_type, int64
- priority, int64
- alloc_collection_id, int64
- instance_index, int64
- machine_id, int64
- resource_request, object
- constraint, object
- collections_events_type, int64
- user, object
- collection_name, object
- collection_logical_name, object
- start_after_collection_ids, object
- vertical_scaling, float64
- scheduler, float64
- start_time, int64
- end_time, int64
- average_usage, object
- maximum_usage, object
- random_sample_usage, object
- assigned_memory, float64
- page_cache_memory, float64
- cycles_per_instruction, float64
- memory_accesses_per_instruction, float64
- sample_rate, float64
- cpu_usage_distribution, object
- tail_cpu_usage_distribution, object
- cluster, int64
- event, object
- failed, object


In [4]:
# First, let's look at a few sample rows to verify the calculation
print("\n=== Sample Duration Calculation ===")
sample_rows = df[['start_time', 'end_time']].head(5)
print("\nSample rows:")
print(sample_rows)

# Calculate duration for sample rows
sample_duration = sample_rows['end_time'] - sample_rows['start_time']
print("\nCalculated durations (in nanoseconds):")
print(sample_duration)

# After verifying, add duration column to full dataset
print("\nAdding duration column to full dataset...")
df['duration'] = df['end_time'] - df['start_time']




=== Sample Duration Calculation ===

Sample rows:
      start_time       end_time
0   274800000000   275100000000
1  1800713000000  1800714000000
2    81300000000    81600000000
3  1075500000000  1075800000000
4  1565315000000  1565317000000

Calculated durations (in nanoseconds):
0    300000000
1      1000000
2    300000000
3    300000000
4      2000000
dtype: int64

Adding duration column to full dataset...


In [5]:
# Remove start_time and end_time columns since we have duration_seconds
print("\nRemoving start_time and end_time columns...")
df = df.drop(['start_time', 'end_time'], axis=1)

print("\nVerifying columns after removal:")
print(df.columns.tolist())



Removing start_time and end_time columns...

Verifying columns after removal:
['Unnamed: 0', 'time', 'instance_events_type', 'collection_id', 'scheduling_class', 'collection_type', 'priority', 'alloc_collection_id', 'instance_index', 'machine_id', 'resource_request', 'constraint', 'collections_events_type', 'user', 'collection_name', 'collection_logical_name', 'start_after_collection_ids', 'vertical_scaling', 'scheduler', 'average_usage', 'maximum_usage', 'random_sample_usage', 'assigned_memory', 'page_cache_memory', 'cycles_per_instruction', 'memory_accesses_per_instruction', 'sample_rate', 'cpu_usage_distribution', 'tail_cpu_usage_distribution', 'cluster', 'event', 'failed', 'duration']


In [7]:
df = df[['average_usage', 'resource_request', 'maximum_usage', 'cpu_usage_distribution', 
         'tail_cpu_usage_distribution', 'assigned_memory', 'page_cache_memory', 
         'duration','vertical_scaling','event','failed']]



In [8]:

# Step 1: Print data types to verify which columns need splitting
print("\nChecking data types of columns:")
print(df.dtypes)

# Step 2: Split average_usage into CPU and memory components
print("\nSplitting average_usage column...")
df['avg_cpu_usage'] = df['average_usage'].apply(lambda x: eval(x)['cpus'])
df['avg_memory_usage'] = df['average_usage'].apply(lambda x: eval(x)['memory'])

# Step 3: Split maximum_usage into CPU and memory components 
print("\nSplitting maximum_usage column...")
df['max_cpu_usage'] = df['maximum_usage'].apply(lambda x: eval(x)['cpus'])
df['max_memory_usage'] = df['maximum_usage'].apply(lambda x: eval(x)['memory'])

# Step 4: Drop the original columns since we've split them
print("\nRemoving original columns...")
df = df.drop(['average_usage', 'maximum_usage'], axis=1)

# Step 5: Verify the new columns
print("\nNew columns after splitting:")
print(df.columns.tolist())

# Step 6: Show sample of new columns to verify data
print("\nSample of split columns:")
print(df[['avg_cpu_usage', 'avg_memory_usage', 'max_cpu_usage', 'max_memory_usage']].head())



Checking data types of columns:
average_usage                   object
resource_request                object
maximum_usage                   object
cpu_usage_distribution          object
tail_cpu_usage_distribution     object
assigned_memory                float64
page_cache_memory              float64
duration                         int64
vertical_scaling               float64
event                           object
failed                          object
dtype: object

Splitting average_usage column...


C:\Users\nithi\AppData\Local\Temp\ipykernel_11748\2305423215.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_cpu_usage'] = df['average_usage'].apply(lambda x: eval(x)['cpus'])
C:\Users\nithi\AppData\Local\Temp\ipykernel_11748\2305423215.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_memory_usage'] = df['average_usage'].apply(lambda x: eval(x)['memory'])



Splitting maximum_usage column...


C:\Users\nithi\AppData\Local\Temp\ipykernel_11748\2305423215.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['max_cpu_usage'] = df['maximum_usage'].apply(lambda x: eval(x)['cpus'])



Removing original columns...

New columns after splitting:
['resource_request', 'cpu_usage_distribution', 'tail_cpu_usage_distribution', 'assigned_memory', 'page_cache_memory', 'duration', 'vertical_scaling', 'event', 'failed', 'avg_cpu_usage', 'avg_memory_usage', 'max_cpu_usage', 'max_memory_usage']

Sample of split columns:
   avg_cpu_usage  avg_memory_usage  max_cpu_usage  max_memory_usage
0       0.004662      5.920410e-03       0.011902      5.935669e-03
1       0.000000      9.536743e-07       0.000000      9.536743e-07
2       0.024200      2.788544e-03       0.060059      2.845764e-03
3       0.047607      3.442383e-02       0.133301      3.466797e-02
4       0.000271      7.629395e-05       0.000415      7.629395e-05


C:\Users\nithi\AppData\Local\Temp\ipykernel_11748\2305423215.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['max_memory_usage'] = df['maximum_usage'].apply(lambda x: eval(x)['memory'])


In [9]:
import ast
def extract_resource_value(x, key):
    if pd.isna(x):
        return np.nan
    try:
        return ast.literal_eval(x)[key]
    except:
        return np.nan

# Split resource_request into requested_cpus and requested_memory
print("\nSplitting resource_request column...")
df['requested_cpus'] = df['resource_request'].apply(lambda x: extract_resource_value(x, 'cpus'))
df['requested_memory'] = df['resource_request'].apply(lambda x: extract_resource_value(x, 'memory'))

# Drop the original resource_request column
df = df.drop('resource_request', axis=1)

# Show sample of new columns
print("\nSample of split columns:")
print(df[['requested_cpus', 'requested_memory']].head())

# Verify the new columns
print("\nVerifying columns after splitting:")
print(df.columns.tolist())

# Save the modified dataset
print("\nSaving modified dataset...")
df.to_csv('final_resource.csv', index=False)







Splitting resource_request column...

Sample of split columns:
   requested_cpus  requested_memory
0        0.020660          0.014435
1        0.007240          0.001303
2        0.048584          0.004166
3        0.070435          0.041626
4        0.002449          0.000232

Verifying columns after splitting:
['cpu_usage_distribution', 'tail_cpu_usage_distribution', 'assigned_memory', 'page_cache_memory', 'duration', 'vertical_scaling', 'event', 'failed', 'avg_cpu_usage', 'avg_memory_usage', 'max_cpu_usage', 'max_memory_usage', 'requested_cpus', 'requested_memory']

Saving modified dataset...


In [13]:
print(df["event"].unique())




['FAIL' 'SCHEDULE' 'FINISH' 'ENABLE' 'EVICT' 'LOST' 'KILL'
 'UPDATE_PENDING' 'QUEUE' 'UPDATE_RUNNING']
